In [29]:
%load_ext autoreload
%autoreload 2

In [13]:
from dotenv import load_dotenv
import os

or_api_key = os.getenv("OR_API_KEY")

In [14]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=or_api_key,
)

In [15]:

def chat_llm(user_prompt, system_prompt="", model="anthropic/claude-sonnet-4"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]
    )
    return completion.choices[0].message.content


In [16]:
def call_llm(messages, model="anthropic/claude-sonnet-4"):
    completion = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return completion.choices[0].message.content

# Methodology

## Value Elicitation

- Select dilemma from dataset
- Chat with the agent
    - Why do you prefer it over the other?
    - Give an example of a related moral situation where you might apply this value
- Extract the value from this conversation. 

## Graph Generation
- Pick two random nodes (how to select nodes?)
- Model generates story of transitioning from Node A to Node B
- User asked if transition is wiser, and if yes, a directed edge is created. Repeat for N edges (can do link prediction? majority voting across jury of models?)
- PageRank usage to rate values


In [17]:
import kagglehub
from kagglehub import KaggleDatasetAdapter


In [18]:
# !curl -L -o reddit.zip \
#   https://www.kaggle.com/api/v1/datasets/download/jianloongliew/reddit

In [19]:
# !unzip reddit.zip -d reddit_data

In [20]:
# import sqlite3

# # Connect to the SQLite database
# conn = sqlite3.connect('reddit_data/AmItheAsshole.sqlite')

In [21]:
# import pandas as pd

# cursor = conn.cursor()
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = cursor.fetchall()

# dfs = {}
# for tbl in tables:
#     tbl_name = tbl[0]
#     dfs[tbl_name] = pd.read_sql_query(f"SELECT * FROM {tbl_name}", conn)

In [22]:
# for key, df_ in dfs.items():
#     df_.to_csv(f"{key}.csv", index=False)

In [23]:
import pandas as pd

In [24]:
submissions_df = pd.read_csv("submission.csv")
# comments_df = pd.read_csv("comment.csv")

In [25]:
submissions_df[:5]

,id,submission_id,title,selftext,created_utc,permalink,score
0,1,xt1ksm,AITA Monthly Open Forum Spooktober 2022,#Keep things civil. Rules still apply.\n\n##Th...,1664646465,/r/AmItheAsshole/comments/xt1ksm/aita_monthly_...,592
1,2,yiplwk,AITA for asking my friend to move a picture of...,"\n\nMe (M32) and my wife, Dahlia (F28) lost ou...",1667251988,/r/AmItheAsshole/comments/yiplwk/aita_for_aski...,16582
2,3,yiv572,AITA for asking my husband to stay with me whi...,Throwaway my family knows my account. I'll get...,1667266450,/r/AmItheAsshole/comments/yiv572/aita_for_aski...,4079
3,4,yimgaf,AITA for telling my SIL to stop talking about ...,My (37M) wife (37F) is pregnant with our first...,1667245059,/r/AmItheAsshole/comments/yimgaf/aita_for_tell...,9728
4,5,yin7pf,"AITA for wanting to meet my ""daughter"" after g...",Long story short: in my (40f) twenties I had a...,1667246573,/r/AmItheAsshole/comments/yin7pf/aita_for_want...,6889


In [26]:
import prompts
print(dir(prompts))

['ANALOGOUS_SITUATIONS', 'GIVE_MORAL_STANCE', 'PAST_SITUATIONS', 'STRESS_TEST_SITUATIONS', 'VALUE_CARD_GENERATION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [27]:
from prompts import GIVE_MORAL_STANCE, ANALOGOUS_SITUATIONS, PAST_SITUATIONS, STRESS_TEST_SITUATIONS, VALUE_CARD_GENERATION

In [28]:
dilemmas = []
for i,j in zip(submissions_df["title"], submissions_df["selftext"]):
    dilemmas.append(f"{i}\n\n{j}")

In [29]:
dilemmas = dilemmas[1:]

In [30]:
i = 2
print(dilemmas[i])

AITA for telling my SIL to stop talking about her miscarriage?

My (37M) wife (37F) is pregnant with our first bio child. We tried to have a baby in our 20s, went through fertility treatments, and nothing worked. This pregnancy was a huge surprise and is a highrisk pregnancy. My wife is 20 weeks now. 

My brother and SIL have 3 kids (aged 5 - 8). SIL was pregnant again this summer and had a miscarriage at 21 weeks. They were obviously devastated and we felt awful for them. 

My SIL talks about her miscarriage a lot, which is understandable. But it has been making my wife really anxious so I first asked my brother if they (he and SIL) can just stop bringing the miscarriage up in front of my wife. He said sure, and stopped, but SIL didn't. So I asked her too, and she said that she will try. She didn't stop. 

Yesterday they came to visit us with the kids to stay a couple of days for Halloween. At lunch, my SIL said to my wife "20 weeks now huh, I remember feeling the baby at 20 weeks. Sa

In [31]:
moral_stance = chat_llm(dilemmas[i], system_prompt=GIVE_MORAL_STANCE)

In [32]:
print(moral_stance)

YTA | While I understand you're trying to protect your wife during a vulnerable time, telling your grieving SIL to "shut the hell up" about her miscarriage was cruel and inappropriate. Your SIL is processing a devastating loss at 21 weeks - which is considered a late pregnancy loss, not just a miscarriage. She's likely still in deep grief.

Instead of handling this with compassion, you could have: had a calm private conversation explaining your wife's anxiety, suggested your wife step away when these topics come up, or found ways to support both women through their difficult experiences. Your wife's anxiety is valid, but so is your SIL's need to process her grief. There were much kinder ways to address this situation that didn't involve attacking a grieving mother.


In [ ]:
def return_situations(dilemma, moral_stance):
    generated_sitations = []
    for prompt in [ANALOGOUS_SITUATIONS, STRESS_TEST_SITUATIONS]:
        messages = [
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": dilemma
            },
            {
                "role": "assistant",
                "content": moral_stance
            }
        ]
        response = call_llm(messages)
        generated_sitations.append(response)
    return generated_sitations

In [34]:
return_situations = return_situations(dilemmas[i], moral_stance)
print(return_situations)

["\n\n**Three similar scenarios:**\n\n1. A cancer survivor repeatedly sharing their treatment stories at family gatherings, causing anxiety for a family member recently diagnosed with cancer, leading to someone angrily telling the survivor to stop talking about their experience.\n\n2. A divorced person frequently discussing their painful divorce proceedings in front of their sibling who is going through marriage counseling, resulting in the sibling's partner harshly telling them to stop bringing up their divorce.\n\n3. An accident victim regularly recounting their trauma and recovery process around a family member with PTSD from a similar incident, causing someone to explosively tell the victim to stop talking about what happened to them.", '\n\n---\n\n**In the crystal mines of Xerion-7, Navigator Zex-9 guides supply ships through treacherous quantum storms to reach isolated colonies. His partner Aria-3 carries their first successful genetic splice after years of failed biotechnology t

In [35]:
print(return_situations[0])



**Three similar scenarios:**

1. A cancer survivor repeatedly sharing their treatment stories at family gatherings, causing anxiety for a family member recently diagnosed with cancer, leading to someone angrily telling the survivor to stop talking about their experience.

2. A divorced person frequently discussing their painful divorce proceedings in front of their sibling who is going through marriage counseling, resulting in the sibling's partner harshly telling them to stop bringing up their divorce.

3. An accident victim regularly recounting their trauma and recovery process around a family member with PTSD from a similar incident, causing someone to explosively tell the victim to stop talking about what happened to them.


In [36]:
print(return_situations[1])



---

**In the crystal mines of Xerion-7, Navigator Zex-9 guides supply ships through treacherous quantum storms to reach isolated colonies. His partner Aria-3 carries their first successful genetic splice after years of failed biotechnology trials - a pregnancy now in its second trimester phase.**

**Zex-9's brother Kale-9 and his mate Nira-5 have three young clones. Last cycle, Nira-5's advanced bioform failed at 21 weeks gestation. She frequently discusses this loss with others, including mentioning how her bioform was developing before the failure.**

**During a recent station visit, Nira-5 told Aria-3 "Second trimester now - I remember the neural connections forming at this stage. Sadly our splice degraded at 21 weeks" causing Aria-3 severe distress. When Zex-9 found Nira-5 continuing these discussions despite requests to avoid the topic around Aria-3, he angrily told her to stop talking about her failed bioform.**

**The other navigators believe Zex-9 was wrong to silence a grie

In [37]:
print(return_situations[2])



The moral principle here is balancing competing needs with compassion rather than silencing someone's grief to protect another's anxiety.

**Three other scenarios where this principle leads to different judgments:**

1. **Workplace scenario**: A coworker keeps discussing their recent divorce during team meetings, making another colleague (going through marriage counseling) uncomfortable. Here, asking for professional boundaries at work would be appropriate since personal trauma shouldn't dominate professional spaces.

2. **Social media scenario**: A friend constantly posts about their pet's death, causing distress to someone whose pet just died. Here, the grieving person could unfollow/mute rather than demanding the poster stop sharing their experience on their own platform.

3. **Support group scenario**: In a pregnancy loss support group, one member asks another to stop sharing detailed loss stories because it triggers their anxiety. Here, the request would be inappropriate since t

In [38]:
from random import shuffle

In [39]:
return_situations.append(dilemmas[i])
shuffle(return_situations)
situations_text = ("\n" + "*" * 50 + "\n").join(return_situations)

In [40]:
situations_system = f"""Below are several situations. All of them encode a specific moral stance. Based on the situations, find a common value that is shared or considered by all of them.\n\n{situations_text}\n\nWhat is the common value that is shared or considered by all of them?.
{VALUE_CARD_GENERATION}"""

In [41]:
chat_llm(situations_system)

'Looking at all these scenarios, I can identify a common moral value that threads through each situation:\n\n```json\n{\n  "value_name": "Contextual Compassion",\n  "description": "The principle that moral responses must balance competing legitimate needs (grief expression vs. psychological protection) while considering the appropriateness of time, place, and relationship dynamics. This value guides decision-making by weighing factors like setting appropriateness, prior requests for accommodation, availability of alternatives, and the severity of harm to all parties involved.",\n  "applications": "Family relationships, workplace boundaries, social media interactions, support groups, medical settings, and any situation where one person\'s coping mechanism directly impacts another\'s wellbeing. The value emphasizes finding solutions that honor both the griever\'s need to process trauma and the vulnerable person\'s need for psychological safety."\n}\n```\n\nThis value recognizes that both

In [42]:
x = """**Three similar scenarios:**

1. A cancer survivor repeatedly sharing their treatment stories at family gatherings, causing anxiety for a family member recently diagnosed with cancer, leading to someone angrily telling the survivor to stop talking about their experience.

2. A divorced person frequently discussing their painful divorce proceedings in front of their sibling who is going through marriage counseling, resulting in the sibling's partner harshly telling them to stop bringing up their divorce.

3. An accident victim regularly recounting their trauma and recovery process around a family member with PTSD from a similar incident, causing someone to explosively tell the victim to stop talking about what happened to them."""